In [2]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

# Step 2: Setup a local conda environment on your machine to run Python and GEE

# Step 3: Install python, geemap, earthengine-api and pandas in your conda environment

# Step 4: 

In [8]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import pandas as pd
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab
# https://geemap.org/notebooks/geemap_and_folium/

import warnings
warnings.filterwarnings('ignore') #don't display warnings

In [9]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'uas_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Temporary Output Folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print(data_dir)

 
****** Temporary Output Folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/uas_output


In [ ]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [10]:
##### Link to Example Cloud Optimized GeoTiffs (COGs) located in a Google Cloud Storage "bucket"
# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_RGB_cog.tif'
# using geemap load the COG file into memory
kg_rgb_image = geemap.load_GeoTIFF(URL)
# create a visualization layer for map display
RGB_imageVisParam = {"opacity":1,"bands":["B0","B1","B2"],
                    "min":-4.063213446967957,"max":85.22120105681381,
                     "gamma":1};

# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_CHM_cog.tif'
kg_chm_image = geemap.load_GeoTIFF(URL)
kg_chm_image = kg_chm_image.select('B0').multiply(0.01)
chm_mask = kg_chm_image.gt(0);
chm_uas_image_masked = kg_chm_image.updateMask(chm_mask);
#print(kg_chm_image.getInfo())
print('CHM Info', kg_chm_image.getInfo()['bands'])
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 2.0, 'palette': dem_palette, 'opacity': 1}

# URL of file in the GCS bucket
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_TIR_cog.tif'
kg_tir_image = geemap.load_GeoTIFF(URL)
kg_tir_image = kg_tir_image.select('B0').multiply(0.1)
tir_mask = kg_tir_image.gt(0);
tir_uas_image_masked = kg_tir_image.updateMask(tir_mask);
tir_palette = cm.get_palette('Spectral_r', n_class=30)
tirViz = {'min': 15, 'max': 30, 'palette': tir_palette, 'opacity': 1}

CHM Info [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [22850, 14148], 'crs': 'EPSG:32603', 'crs_transform': [0.01, 0, 508169.23, 0, -0.010000000000031596, 7226835.705]}]


In [11]:
##### Create a simple map displaying an RGB UAS image (Yang et al., 2021)

# center map over the Kougarok study site
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
uas_map # creates a geemap.folium layer

In [12]:
##### Create a simple map displaying a Canopy Height Model (CHM) image for the same UAS flight (Yang et al., 2021)

# center map over Kougarok
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')
uas_map.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                     vmax=demViz['max'], label="SfM CHM (meters)", 
                     caption = "SfM CHM (meters)",
                     layer_name="SfM CHM (meters)", position="bottomright")
uas_map

In [13]:
##### Create a simple map displaying a Thermal Infrared (TIR) image for the same UAS flight (Yang et al., 2021)

uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
uas_map.addLayer(tir_uas_image_masked, tirViz, 'UAS_Kougarok_20180725_Flight6_TIR')
uas_map.add_colorbar(colors=tirViz['palette'], vmin=tirViz['min'], 
                     vmax=tirViz['max'], label="LST (deg C)", 
                     caption = "LST (deg C)",
                     layer_name="LST (deg C)", position="bottomright")
uas_map.addLayerControl()
uas_map

In [15]:
### Combined map
# center map over Kougarok
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=18);
uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')
uas_map.add_colorbar(colors=demViz['palette'], vmin=demViz['min'], 
                     vmax=demViz['max'], label="SfM CHM (meters)", 
                     caption = "SfM CHM (meters)",
                     layer_name="SfM CHM (meters)", position="bottomright")
uas_map.addLayer(tir_uas_image_masked, tirViz, 'UAS_Kougarok_20180725_Flight6_TIR')
uas_map.add_colorbar(colors=tirViz['palette'], vmin=tirViz['min'], 
                     vmax=tirViz['max'], label="LST (deg C)", 
                     caption = "LST (deg C)",
                     layer_name="LST (deg C)", position="bottomright")
uas_map

In [16]:
### Combining UAS and other datasets - in this case GEE datasets
elev_dataset = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic')
# https://developers.google.com/earth-engine/datasets/catalog/UMN_PGC_ArcticDEM_V3_2m_mosaic
ned_elevation = elev_dataset.select('elevation')
dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz2 = {'min': 0.0, 'max': 150, 'palette': dem_palette, 'opacity': 1}

# Global Forest Canopy Height (2005)
gfch = ee.Image('NASA/JPL/global_forest_canopy_height_2005');
forestCanopyHeight = gfch.select('1');
gfch_palette = cm.get_palette('Spectral_r', n_class=30)
gfchViz = {'min': 0.0, 'max': 40.0, 'palette': gfch_palette, 'opacity': 1}


uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=15);
uas_map.addLayer(forestCanopyHeight, gfchViz, 'Global Forest Canopy Height (2005, in meters)')
uas_map.add_colorbar(colors=gfchViz['palette'], vmin=gfchViz['min'], vmax=gfchViz['max'],
                     label='Global Forest Canopy Height (2005, in meters)', 
                     layer_name='Global Forest Canopy Height', 
                     caption = 'Global Forest Canopy Height (2005, in meters)',
                     position='bottomright', orientation='vertical')
uas_map.addLayer(ned_elevation, demViz2, 'Elevation above sea level');
uas_map.add_colorbar(colors=demViz2['palette'], vmin=demViz2['min'], 
                     vmax=demViz2['max'], label="Elevation (m)", 
                     caption = "Elevation (m)", 
                     layer_name="ArcticDEM", position="bottomright")


uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
uas_map

In [18]:
### Extract random points through CHM layer
# !NOTE: this step tends to take awhile as the data is extracted from the remote file
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')

region = ee.Geometry.Point([-164.8239, 65.1644]).buffer(30)
uas_map.addLayer(ee.Image().paint(region, 0, 4), {'palette': 'yellow'}, "ROI-1")

random_points = geemap.random_sampling(chm_uas_image_masked, region, numPixels=100)
uas_map.addLayer(random_points, {}, "Random Sampling")

uas_map

In [ ]:
### Show results of random sampling (x100)
df = geemap.ee_to_pandas(random_points)
print("SfM CHM in meters", df)